In [1]:
import cv2
import torchvision
import numpy as np
from keras.preprocessing.image import load_img
import torchvision.transforms as T


model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

img_path = '2007_000032.jpg'
img = load_img(img_path, target_size=(800, 800))
transform = T.Compose([T.ToTensor()])  # Defing PyTorch Transform
img = transform(img)  # Apply the transform to the image

images = [img]
targets=None
original_image_sizes = [img.shape[-2:] for img in images]
images, targets = model.transform(images, targets)

# 提取特征
features = model.backbone(images.tensors)

#提取proposals
proposals, proposal_losses = model.rpn(images, features, targets)


Using TensorFlow backend.


以其中一个proposal [257.4335, 268.6282, 648.8815, 444.5958]为例，这个proposal缩放（stride=16）后是 [16.0896, 16.7893, 40.5551, 27.7872], 分别对应roi左上角的x，y坐标、roi右下角的x，y坐标

In [2]:
import torch

rois_in_feature = torch.Tensor([[0, 16.0896, 16.7893, 40.5551, 27.7872]])

In [3]:
height = 50
width = 50
pooled_height = 7
pooled_width = 7

roi_start_w = 16.0896
roi_start_h = 16.7893
roi_end_w = 40.5551
roi_end_h = 27.7872

In [4]:
roi_width = max((roi_end_w - roi_start_w), 1)
roi_height = max((roi_end_h - roi_start_h), 1)
bin_size_h = roi_height / pooled_height
bin_size_w = roi_width / pooled_width

In [5]:
roi_bin_grid_h = 2
roi_bin_grid_w = 2
count = 4

In [8]:
per_level_feature = features[2]

In [9]:
for h in range(pooled_height): 
    for w in range(pooled_width):
        res = 0
        for i in range(2):
            for j in range(2):
                point_x = roi_start_w + w*bin_size_w + (bin_size_w*(1+2*i)/4)
                point_y = roi_start_h + h*bin_size_h + (bin_size_h*(1+2*j)/4)
        
                val0 = per_level_feature[0][0][int(point_y)][int(point_x)]
                val1 = per_level_feature[0][0][int(point_y)][int(point_x)+1]
                val2 = per_level_feature[0][0][int(point_y)+1][int(point_x)]
                val3 = per_level_feature[0][0][int(point_y)+1][int(point_x)+1]

                x1 = point_x - int(point_x)
                x2 = int(point_x) + 1 - point_x
                y1 = point_y - int(point_y)
                y2 = int(point_y) + 1 - point_y

                area0 = x1 * y1
                area1 = x2 * y1
                area2 = x1 * y2
                area3 = x2 * y2

                res += (val0*area3 + val1*area2 + val2*area1 + val3*area0)

        print(res/4)

tensor(0.1315, grad_fn=<DivBackward0>)
tensor(0.3561, grad_fn=<DivBackward0>)
tensor(0.1099, grad_fn=<DivBackward0>)
tensor(0.0105, grad_fn=<DivBackward0>)
tensor(0.1069, grad_fn=<DivBackward0>)
tensor(0.4987, grad_fn=<DivBackward0>)
tensor(-0.0353, grad_fn=<DivBackward0>)
tensor(0.5771, grad_fn=<DivBackward0>)
tensor(0.1737, grad_fn=<DivBackward0>)
tensor(0.0971, grad_fn=<DivBackward0>)
tensor(-0.0943, grad_fn=<DivBackward0>)
tensor(-0.0527, grad_fn=<DivBackward0>)
tensor(0.6988, grad_fn=<DivBackward0>)
tensor(0.3200, grad_fn=<DivBackward0>)
tensor(-0.1067, grad_fn=<DivBackward0>)
tensor(-0.4572, grad_fn=<DivBackward0>)
tensor(-0.3240, grad_fn=<DivBackward0>)
tensor(-0.5574, grad_fn=<DivBackward0>)
tensor(-0.3299, grad_fn=<DivBackward0>)
tensor(0.5116, grad_fn=<DivBackward0>)
tensor(0.3760, grad_fn=<DivBackward0>)
tensor(-0.5320, grad_fn=<DivBackward0>)
tensor(-0.6477, grad_fn=<DivBackward0>)
tensor(-0.8813, grad_fn=<DivBackward0>)
tensor(-0.8584, grad_fn=<DivBackward0>)
tensor(-0.592

In [11]:
from torchvision.ops import roi_align

result = roi_align(
                per_level_feature, rois_in_feature,
                (7, 7), sampling_ratio=2
            )

In [12]:
result[0][0]

tensor([[ 0.1315,  0.3561,  0.1099,  0.0105,  0.1069,  0.4987, -0.0353],
        [ 0.5771,  0.1737,  0.0971, -0.0943, -0.0527,  0.6988,  0.3200],
        [-0.1067, -0.4572, -0.3240, -0.5574, -0.3299,  0.5116,  0.3760],
        [-0.5320, -0.6477, -0.8813, -0.8584, -0.5922,  0.2852,  0.3046],
        [-0.3979, -0.1485, -0.5802, -0.2516, -0.3406,  0.3955,  0.3633],
        [ 0.1361, -0.0142,  0.1989,  0.8769,  0.2482,  0.3156,  0.4014],
        [ 0.6164,  0.3802,  0.7231,  0.8693,  0.3949,  0.4931,  0.9423]],
       grad_fn=<SelectBackward>)